<a href="https://colab.research.google.com/github/jeremysb1/grokking-ml/blob/main/adaline_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptive linear neurons and the convergence of learning.

I am taking a look at a type of single-layer neural network (NN): ADAptive LInear NEuron (Adaline).

The Adaline algorithm is particularly interesting because it illustrates the key concepts of defining and minimizing continuous loss functions.

## Minimizing loss functions with gradient decent

One of the key ingredients of supervised machine learning algorithms is a defined objective function that is to be optimized during the learning process. This objective function is often a loss or cost function that we want to minimize.

In the case of Adaline, we can define the loss function, L, to learn the model parameters as the mean squared error (MSE) between the calculated outcome and the true class label.



In [1]:
import os
import numpy as np
import pandas as pd


In [ ]:
class AdalineGD:

  """ADAptive LInear NEuron classifier.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.
    random_state : int
        Random number generator seed for random weight initialization.

    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    b_ : Scalar
        Bias unit after fitting.
    losses_ : list
      Mean squared error loss function values in each epoch.
    """

